In [1]:
from plio.io import io_controlnetwork
from knoten.csm import create_csm
import ale
import os

/home/arsanders/miniconda3/envs/knoten/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


In [2]:
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
import pysis
from pysis import isis

In [4]:
df = io_controlnetwork.from_isis('data/hand_dense.net')

In [5]:
df

,id,pointType,pointChoosername,pointDatetime,pointEditLock,pointIgnore,pointJigsawRejected,referenceIndex,aprioriSurfPointSource,aprioriSurfPointSourceFile,...,measureDatetime,measureEditLock,measureIgnore,measureJigsawRejected,diameter,apriorisample,aprioriline,samplesigma,linesigma,measureLog
0,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,201.356777,302.569131,0.0,0.0,[]
1,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,754.094710,15168.130157,0.0,0.0,[]
2,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,399.936010,13312.074608,0.0,0.0,[]
3,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,1926.464206,96.383931,0.0,0.0,[]
4,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,2100.233957,14938.826738,0.0,0.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,...,2019-12-11T16:05:47,False,False,False,0.0,2413.917268,22462.671510,0.0,0.0,[]
126,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,...,2019-12-11T16:05:47,False,False,False,0.0,2445.984758,24263.903615,0.0,0.0,[]
127,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,...,2019-12-12T08:09:54,False,False,False,0.0,3843.915024,24115.819511,0.0,0.0,[]
128,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,...,2019-12-12T08:09:54,False,False,False,0.0,4009.638555,9314.698759,0.0,0.0,[]


In [12]:
isd_files = []
sensors = {}
for line in open('data/cubes.lis'):
    basename = os.path.splitext(os.path.basename(line.strip()))[0]
    isd = os.path.join('data',basename+'.json')
    isd_files.append(isd)
    with open(isd, 'w+') as f:
        f.write(ale.loads(line.strip(), formatter='usgscsm'))
        
    sn = isis.getsn(from_=line.strip()).strip().decode('utf-8')
    sensors[sn] = create_csm(isd)


In [14]:
for sn, sensor in sensors.items():
    sensor.getReferencePoint()